# Automate feature engineering process

# Define necessary functions and classes

In [1]:
# class for imputing missing values on each group
# group by key: groupby_columns
# column to be imputed: impute_column

# group_by_imputer = GroupByImputer(strategy="median")
# group_by_imputer.fit(data_train, groupby_columns=['Pclass', 'Sex'], impute_column='Age')
# data_train = group_by_imputer.transform(data_train)
# data_test = group_by_imputer.transform(data_test)

class GroupByImputer():
    def __init__(self, missing_values="NaN", strategy="mean", 
                 axis=0, verbose=0, copy=True):
        self.missing_values = missing_values
        self.strategy = strategy
        self.axis = axis
        self.verbose = verbose
        self.copy = copy
    
    def fit(self, X, groupby_columns, impute_column, y=None):
        
        self.group_by_imputers_ = X.groupby(groupby_columns).apply(lambda x: self._imputer_fit(x, impute_column, self.strategy))
        
        self.groupby_columns_ = groupby_columns
        self.impute_column_ = impute_column
        
        return self
    
    def _imputer_fit(self, x, impute_column, strategy):
        
        if data_train[impute_column].dtype == np.dtype('O'): 
            # object string
            imputer = x[impute_column].value_counts().index[0]
        else:
            # int or float
            imputer = preprocessing.Imputer(strategy=strategy)
            imputer.fit(x[[impute_column]])
        
        return imputer
    
    
    def transform(self, X):
        X = X.groupby(self.groupby_columns_).apply(lambda x: self._imputer_transform(x))
        return X
    
    def _imputer_transform(self, x):
        
        index = x.name
        imputer = self.group_by_imputers_[index]
        
        if data_train[self.impute_column_].dtype == np.dtype('O'): 
            # object string
            x[[self.impute_column_]] = x[[self.impute_column_]].fillna(imputer)
            
        else:
            # int or float
            x[[self.impute_column_]] = imputer.transform(x[[self.impute_column_]])
        
        return x

## 1. Start

### Import Modules

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import OrderedDict

from sklearn import preprocessing

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC, LinearSVC

from sklearn.cross_validation import KFold
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score

import category_encoders as ce

import xgboost as xgb

from bayes_opt import BayesianOptimization

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Read Data

In [3]:
data_train = pd.read_csv('./data/train.csv')
data_test = pd.read_csv('./data/test.csv')

In [4]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
data_train.describe(percentiles=[0.2,0.8], include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Johnson, Miss. Eleanor Ileen",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
20%,179.000000,0.000000,1.000000,NaN,NaN,19.000000,0.000000,0.000000,NaN,7.854200,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
80%,713.000000,1.000000,3.000000,NaN,NaN,41.000000,1.000000,1.000000,NaN,39.687500,NaN,NaN


## 2. Preprocessing

### Missing Values

In [6]:
## TODO: MICE implementation

In [7]:
# columns with missing values
columns_with_missing_values = list(data_train.columns[data_train.isnull().any()])
# number of missing values on each column
print(data_train[columns_with_missing_values].isnull().sum())

Age         177
Cabin       687
Embarked      2
dtype: int64


In [8]:
# Impute missing values with median value on each (Pclass, Sex) group for Age column
age_group_by_imputer = GroupByImputer(strategy="median")
age_group_by_imputer.fit(data_train, groupby_columns=['Pclass', 'Sex'], impute_column='Age')
data_train = age_group_by_imputer.transform(data_train)

In [9]:
# Impute missing values with median value on each (Pclass, Embarked) group for Fare column
fare_group_by_imputer = GroupByImputer(strategy="median")
fare_group_by_imputer.fit(data_train, groupby_columns=['Pclass', 'Embarked'], impute_column='Fare')

In [10]:
# Impute missing values for Embarked column
# Hypothesis: Fare is positive correlated with Pclass and Embarked

In [11]:
data_train[['Pclass', 'Embarked', 'Fare']].groupby(['Pclass', 'Embarked']).median()

Fare
Pclass Embarked         
1      C         78.2667
       Q         90.0000
       S         52.0000
2      C         24.0000
       Q         12.3500
       S         13.5000
3      C          7.8958
       Q          7.7500
       S          8.0500

In [12]:
data_train.loc[data_train['Embarked'].isnull(), :]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [13]:
# For Pclass 1, Fare 80 is nearest to Fare 78.2667, which Embarked is C
data_train.loc[data_train['Embarked'].isnull(), 'Embarked'] = 'C'

In [14]:
# # Handling on Cabin: 0 for null and 1 for not null
# data_train['Cabin_flg'] = 0
# data_train.loc[data_train.Cabin.notnull(), 'Cabin_flg'] = 1

### Feature Transformation

In [15]:
# # Convert SibSp with more than 2 into 'more_than_2'
# data_train['SibSp_flg'] = data_train['SibSp']
# data_train.loc[data_train['SibSp'] >= 2, 'SibSp_flg'] = 'more_than_2'
# data_train['SibSp_flg'] = data_train['SibSp_flg'].astype(str)

In [16]:
# # Convert Parch with more than 2 into 'more_than_2'
# data_train['Parch_flg'] = data_train['Parch']
# data_train.loc[data_train['Parch'] >= 2, 'Parch_flg'] = 'more_than_2'
# data_train['Parch_flg'] = data_train['Parch_flg'].astype(str)

### Feature Generation

In [17]:
# Extract Title from Name
def generate_title(data, column='Name', new_column='Title', main_levels=['Mr', 'Master', 'Miss', 'Mrs']):
    # extract title from name
    data[new_column] = data[column].str.extract(',\s*([^\.]*)\s*\.', expand=False)
    # Mr: Mr; Master: Master; Miss: Miss; Mrs: Mrs; Others: Rare
    data[new_column] = data[new_column].str.replace('|'.join(list(set(data[new_column].unique()) - set(main_levels))), 'Rare')
    return data

data_train = generate_title(data_train)

In [18]:
# Combine SibSp and Parch and passenger as Fsize(Family Size)
data_train['Fsize'] = data_train['SibSp'] + data_train['Parch'] + 1

In [19]:
# Create new feature to divide Age into 'child' and 'adult'
data_train['Age_New'] = 'adult'
data_train.loc[data_train['Age'] < 18, 'Age_New'] = 'child'

### Preparation for Modeling

In [20]:
# Feature list
numerical_variables = ['Fsize', 'Fare']
categorical_variables = ['Pclass', 'Sex', 'Embarked', 'Title', 'Age_New']
# features = numerical_variables + categorical_variables

In [21]:
# OneHotEncoder
ohe = ce.one_hot.OneHotEncoder(cols=categorical_variables)
ohe.fit(data_train)
# Transform training data
data_train = ohe.transform(data_train)
# Remove columns with name containing '-1'(all 0)
data_train = data_train[[c for c in data_train.columns if '-1' not in c]]

### Train Models

In [22]:
# Dummy categorical variables created by OneHotEncoding
dummy_categorical_variables = list()
for categorical_variable in categorical_variables:
    dummy_categorical_variables = dummy_categorical_variables + [c for c in data_train.columns if categorical_variable in c]
    
features = dummy_categorical_variables + numerical_variables

In [23]:
X_train = data_train[features]
y_train = data_train['Survived']

In [24]:
classifiers = OrderedDict()
classifiers['Logistic Regression'] = LogisticRegression()
classifiers['Decision Tree'] = DecisionTreeClassifier()
classifiers['Random Forest'] = RandomForestClassifier()
classifiers['AdaBoost'] = AdaBoostClassifier()
classifiers['Gradient Boosting'] = GradientBoostingClassifier()
classifiers['Naive Bayes'] = GaussianNB()
classifiers['XGBoost'] = xgb.XGBClassifier()
# 各学習器をCVでパフォーマンスを出力
cv_result_df = pd.DataFrame(columns=['classifier', 'cv_scores_mean'])
for clf_name, classifier in classifiers.items():
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=5)
    cv_result_df = cv_result_df.append({cv_result_df.columns.values[0]: clf_name,\
                                        cv_result_df.columns.values[1]: cv_scores.mean()},\
                                       ignore_index=True)

In [25]:
cv_result_df

,classifier,cv_scores_mean
0,Logistic Regression,0.824943
1,Decision Tree,0.806997
2,Random Forest,0.804806
3,AdaBoost,0.818233
4,Gradient Boosting,0.824981
5,Naive Bayes,0.782290
6,XGBoost,0.807016


### Parameter Tuning

In [ ]:
# XGBoost

In [26]:
## Grid Search

In [39]:
clf = xgb.XGBClassifier(learning_rate=0.1, silent=True, objective='binary:logistic')

param_grid = {
    'n_estimators': list(range(100, 200, 10)),
    'max_depth': list(range(3, 10, 1)),
    'subsample': [0.8, 0.9, 1]
}

grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, scoring='accuracy', cv=5)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 110, 120, 130, 140, 150, 160, 170, 180, 190], 'max_depth': [3, 4, 5, 6, 7, 8, 9], 'subsample': [0.8, 0.9, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [40]:
grid_search.grid_scores_, grid_search.best_params_, grid_search.best_score_

([mean: 0.82828, std: 0.02808, params: {'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8},
  mean: 0.82267, std: 0.02800, params: {'max_depth': 3, 'n_estimators': 100, 'subsample': 0.9},
  mean: 0.80696, std: 0.03328, params: {'max_depth': 3, 'n_estimators': 100, 'subsample': 1},
  mean: 0.82604, std: 0.02751, params: {'max_depth': 3, 'n_estimators': 110, 'subsample': 0.8},
  mean: 0.81930, std: 0.03019, params: {'max_depth': 3, 'n_estimators': 110, 'subsample': 0.9},
  mean: 0.81481, std: 0.03089, params: {'max_depth': 3, 'n_estimators': 110, 'subsample': 1},
  mean: 0.82492, std: 0.02363, params: {'max_depth': 3, 'n_estimators': 120, 'subsample': 0.8},
  mean: 0.82379, std: 0.02863, params: {'max_depth': 3, 'n_estimators': 120, 'subsample': 0.9},
  mean: 0.81369, std: 0.03155, params: {'max_depth': 3, 'n_estimators': 120, 'subsample': 1},
  mean: 0.82941, std: 0.02561, params: {'max_depth': 3, 'n_estimators': 130, 'subsample': 0.8},
  mean: 0.82155, std: 0.03134, params: {'max_d

In [44]:
## BayesianOptimization

In [35]:
clf = xgb.XGBClassifier(learning_rate=0.1, silent=True, objective='binary:logistic')

# def xgb_evaluate(n_estimators, max_depth, subsample): 
#     
#     params['n_estimators'] = int(n_estimators)
#     params['max_depth'] = int(max_depth)
#     params['subsample'] = max(min(subsample, 1), 0)
#     
#     cv_result = xgb.cv(params, xgtrain, num_boost_round=num_rounds, nfold=5,
#              seed=random_state,
#              callbacks=[xgb.callback.early_stop(50)])
#     
#     return -cv_result['test-mae-mean'].values[-1]

# params = {
#     'eta': 0.1,
#     'silent': 1,
#     'eval_metric': 'mae',
#     'verbose_eval': True,
#     'seed': random_state
# }

def xgb_evaluate(n_estimators, max_depth, subsample): 
    
    n_estimators= int(n_estimators)
    max_depth= int(max_depth)
    subsample = max(min(subsample, 1), 0)
    
    clf = xgb.XGBClassifier(learning_rate=0.1, n_estimators=n_estimators, \
                            max_depth=max_depth, subsample=subsample, \
                            silent=True, objective='binary:logistic')
    
    cv_scores = cross_val_score(clf, X_train, y_train, cv=5)
    
    return cross_val_score(clf, X_train, y_train, cv=5).mean()




xgbBO = BayesianOptimization(xgb_evaluate, {'n_estimators': (100, 200),
                                            'max_depth': (3, 10),
                                            'subsample': (0.5, 1)
                                            })
num_iter = 25
init_points = 5

xgbBO.maximize(init_points=init_points, n_iter=num_iter)

Initialization
--------------------------------------------------------------------------
 Step |   Time |      Value |   max_depth |   n_estimators |   subsample | 
    1 | 00m02s |    0.82388 |      9.4978 |       185.2594 |      0.9748 | 
    2 | 00m01s |    0.82723 |      9.0468 |       133.6669 |      0.5692 | 
    3 | 00m00s |    0.82722 |      3.5421 |       155.3604 |      0.5864 | 
    4 | 00m00s |    0.82611 |      3.0321 |       190.5833 |      0.6820 | 
    5 | 00m01s |    0.82722 |      7.5477 |       187.1834 |      0.6955 | 
Bayesian Optimization
--------------------------------------------------------------------------
 Step |   Time |      Value |   max_depth |   n_estimators |   subsample | 


/opt/conda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([  1.50904054e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/opt/conda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -6.02762752e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


    6 | 00m20s |    0.82161 |      3.0000 |       100.0000 |      0.5000 | 


/opt/conda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


    7 | 00m15s |    0.83174 |      9.9479 |       199.9642 |      0.5329 | 


...

   28 | 00m15s |    0.83174 |      9.9564 |       140.9409 |      0.9997 | 


/opt/conda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   29 | 00m14s |    0.83173 |      6.6020 |       100.0170 |      0.6700 | 


/opt/conda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -6.41721815e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 65, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/opt/conda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   30 | 00m16s |    0.82160 |      3.1253 |       131.3381 |      0.9746 | 


In [37]:
xgbBO.res['max']['max_params']

{'max_depth': 9.9748670659455332,
 'n_estimators': 100.15571535878696,
 'subsample': 0.53975673408943159}

In [36]:
xgbBO.res['max']['max_val']

0.83400111426856005

In [44]:
# grid_search
clf = grid_search.best_estimator_

# # bayesian optimization
# clf = xgb.XGBClassifier(max_depth=int(xgbBO.res['max']['max_params']['max_depth']), \
#                        n_estimators=int(xgbBO.res['max']['max_params']['n_estimators']), \
#                        subsample=xgbBO.res['max']['max_params']['n_estimators'])

In [45]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=190, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

### Transformation on Test Data

In [46]:
# Append new column of tartget column
data_test['Survived'] = 0

In [47]:
# Impute missing values with median value on each (Pclass, Sex) group for Age column
data_test = age_group_by_imputer.transform(data_test)

In [48]:
# Impute missing values with median value on each (Pclass, Embarked) group for Fare column
data_test = fare_group_by_imputer.transform(data_test)

In [49]:
# Combine SibSp and Parch and passenger as Fsize(Family Size)
data_test['Fsize'] = data_test['SibSp'] + data_test['Parch'] + 1

In [50]:
# Extract Title from Name
data_test = generate_title(data_test)

In [51]:
# Create new feature to divide Age into 'child' and 'adult'
data_test['Age_New'] = 'adult'
data_test.loc[data_test['Age'] < 18, 'Age_New'] = 'child'

In [52]:
# Transform test data
data_test = ohe.transform(data_test)
# Remove columns with name containing '-1'(all 0)
data_test = data_test[[c for c in data_test.columns if '-1' not in c]]

In [53]:
# for categorical_column in categorical_variables:
#     lb = LabelBinarizerDict[categorical_column]
#     
#     # dummy dataframe 
#     dummy_df = pd.DataFrame(lb.transform(data_test[categorical_column]), index=None)
#     
#     # column names of dummy variables
#     dummy_column_names = [categorical_column + '_' + str(lb_class) for lb_class in list(lb.classes_)]
#     if len(dummy_column_names) == 2:
#         dummy_column_names = [dummy_column_names[0]]
#     # assign column names to dummy dataframe
#     dummy_df.columns = dummy_column_names
#     
#     data_test = pd.concat([data_test, dummy_df], axis=1)

In [54]:
X_test = data_test[dummy_categorical_variables + numerical_variables]

In [55]:
# random forest
y_test = clf.predict(X_test)

In [56]:
## submit the result
pd.DataFrame({'PassengerId':data_test.PassengerId, 'survived':y_test}).to_csv('solution.csv', header=True, index=False)